In [1]:
import pandas as pd
import dtale
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import plotly.graph_objects as go # for visualization
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import os
import statsmodels.api as sm
import sys 
sys.path.append("../../Function/")
from function import remove_high_corr
from function import remove_outliers

%load_ext nb_black
%load_ext blackcellmagic

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


<IPython.core.display.Javascript object>

In [86]:
df = pd.read_excel("../../../site_info_ver_3.2.xlsx")



<IPython.core.display.Javascript object>

In [87]:
df = df[df.is_weekend==1]
df

,Date,Model_number,Site_Name,Israelis_Count,Tourists_Count,Total,region_Central,region_Judea_Samaria,region_North,region_South,is_weekend,operations,is_jewish_holiday,is_jewish_vacation,is_muslims_holiday,is_muslims_vacation,is_camping,special_activity,visit_duration,is_jewishHeritage,is_muslimsHeritage,is_christiansHeritage,is_nationalPark,is_natureReserve,is_archaeology,is_religion,is_animals,is_heritage,is_water,is_cave,is_lookout,pm10,pm2.5,nox,so2,Temperature,is_HeatWave,Tel_Aviv-Yafo_pm10,Jerusalem_pm10,Haifa_pm10,Ashkelon_pm10,Beer-Sheva_pm10,Tel_Aviv-Yafo_pm2.5,Jerusalem_pm2.5,Haifa_pm2.5,Ashkelon_pm2.5,Beer-Sheva_pm2.5,Tel_Aviv-Yafo_nox,Jerusalem_nox,Haifa_nox,Ashkelon_nox,Beer-Sheva_nox,Tel_Aviv-Yafo_so2,Jerusalem_so2,Haifa_so2,Ashkelon_so2,Beer-Sheva_so2,is_Site_exceeded_pm10,is_Site_exceeded_pm2.5,is_Site_exceeded_nox,is_Site_exceeded_so2,Tel_Aviv-Yafo_pm10_exceeded,Jerusalem_pm10_exceeded,Haifa_pm10_exceeded,Ashkelon_pm10_exceeded,Beer-Sheva_pm10_exceeded,Tel_Aviv-Yafo_pm2.5_exceeded,Jerusalem_pm2.5_exceeded,Haifa_pm2.5_exceeded,Ashkelon_pm2.5_exceeded,Beer-Sheva_pm2.5_exceeded,Tel_Aviv-Yafo_so2_exceeded,Jerusalem_so2_exceeded,Haifa_so2_exceeded,Ashkelon_so2_exceeded,Beer-Sheva_so2_exceeded,Tel_Aviv-Yafo_nox_exceeded,Jerusalem_nox_exceeded,Haifa_nox_exceeded,Ashkelon_nox_exceeded,Beer-Sheva_nox_exceeded,Green_border,Season_autumn,Season_spring,Season_summer,Season_winter
0,2016-01-01,5,Alexander Stream-Bet Yannai,7,0,7,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,8.0,NaN,6.20,0.6,15.5,0,64.1,25.6,23.3,11.2,4.9,13.9,30.1,26.2,4.8,2.5,69.1,501.7,90.2,3.8,9.7,7.2,0.1,2.6,1.2,0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1
1,2016-01-02,5,Alexander Stream-Bet Yannai,28,0,28,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,23.5,NaN,66.10,7.5,13.8,0,154.1,277.1,50.6,26.0,51.4,87.1,80.7,93.9,21.6,36.0,749.9,1140.2,842.8,10.8,115.7,16.7,1.9,20.0,8.8,2.3,0,0,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
7,2016-01-08,5,Alexander Stream-Bet Yannai,14,0,14,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,25.80,16.1,20.5,0,80.8,303.2,82.3,45.3,49.3,44.6,73.6,123.1,27.7,25.5,120.8,1152.4,208.7,5.4,37.8,8.0,0.2,2.3,0.8,1.5,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
8,2016-01-09,5,Alexander Stream-Bet Yannai,71,0,71,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,36.7,NaN,22.97,0.0,18.2,0,90.9,224.5,24.8,23.7,38.8,49.1,63.5,89.8,26.3,20.2,166.9,1268.1,225.1,2.2,37.1,7.6,0.9,6.3,2.2,2.2,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
14,2016-01-15,5,Alexander Stream-Bet Yannai,64,0,64,1,0,0,0,1,0,0,0,0,0,0,0,3,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,57.80,3.9,19.5,0,90.7,105.3,30.0,13.8,13.7,20.3,57.1,25.9,9.7,7.1,338.0,664.9,451.1,4.5,48.3,8.5,0.3,3.2,1.2,1.6,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109098,2021-07-03,2,Baptismal Site Qasr al-Yahud,2,0,2,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,378.5,57.9,720.90,NaN,38.5,1,133.7,378.5,95.0,27.2,30.0,162.8,57.9,109.3,17.7,17.3,1397.4,720.9,356.5,11.4,40.7,9.3,2.9,15.6,2.8,4.4,1,1,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,1,1,0,0,1,0
109100,2021-07-09,2,Baptismal Site Qasr al-Yahud,5,0,5,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,30.5,7.5,2.60,2.4,38.5,1,121.6,117.9,102.0,29.9,25.6,138.0,47.5,109.6,17.2,11.9,111.7,49.1,60.3,1.5,15.6,8.7,2.9,8.5,2.0,3.7,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,1,0
109101,2021-07-10,2,Baptismal Site Qasr al-Yahud,8,0,8,0,1,0,0,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,31.8,12.5,26.90,NaN,38.5,1,138.2,431.3,115.1,41.1,40.8,118.9,77.5,110.0,23.8,12.1,1118.4,902.9,415.0,5.7,46.1,13.0,3.6,17.8,1.8,5.9,0,0,0,0,1,1,0

<IPython.core.display.Javascript object>

Check Null

In [88]:
# drops = remove_high_corr(df, target, threshold)
# df.isna().sum()
# df=df.dropna()
# df.reset_index(inplace=True,drop=True)
# df.drop(['pm10','nox','so2','pm2.5'],axis=1,inplace=True)
len(df[df.isnull().any(axis=1)])/len(df)

0.5720049559996188

<IPython.core.display.Javascript object>

Defiend X and y

In [89]:
X = df.drop(['Israelis_Count','Tourists_Count','Total',"Date","Model_number","is_weekend"],axis=1)
targets = df[['Israelis_Count','Tourists_Count','Total']]

# X_dummies = pd.get_dummies(X,'Site_Name',drop_first=True)
targets

,Israelis_Count,Tourists_Count,Total
0,7,0,7
1,28,0,28
7,14,0,14
8,71,0,71
14,64,0,64
...,...,...,...
109098,2,0,2
109100,5,0,5
109101,8,0,8
109103,5,0,5


<IPython.core.display.Javascript object>

Scaleing

In [90]:

#scale X
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X =X.drop("Site_Name",axis=1)
scaled_X = pd.DataFrame(scaler.fit_transform(scaled_X), columns=scaled_X.columns)
scaled_X["Site_Name"] = X.Site_Name

#scale y
scaled_targets = np.log(targets+0.01)
scaled_targets

scaled_df = scaled_X.merge(scaled_targets,left_index=True,right_index=True)

<IPython.core.display.Javascript object>

Model MLR with Pollutions droped Null rows

In [109]:
treshold = 0.8
Descripton = "Only weekend corr drop with treshold "+str(treshold)+" remove nulls rows"# and outliers" 
Model_type= "MLR"

res=pd.read_excel("../../../res.xlsx")

# targets = [scaled_targets.Israelis_Count,scaled_targets.Total,scaled_targets.Tourists_Count]
targets_list_name = ['Israelis_Count','Total','Tourists_Count']
for idx in range(3):
    tmp = scaled_df.copy()
    target_drop=[x for x in targets_list_name if x!=targets_list_name[idx]]
    tmp.drop(target_drop,axis=1,inplace=True)
    drops = remove_high_corr(tmp.drop("Site_Name",axis=1), targets_list_name[idx],treshold)
    tmp = tmp.drop(drops,axis=1)
    tmp.dropna(inplace=True)
    # tmp = remove_outliers(tmp, targets_list_name[idx])
    y = tmp[targets_list_name[idx]]
    X = tmp.drop(targets_list_name[idx],axis=1)
    #split the data, train the model and get prediction for the training and for the test  
    #######################################################################################################################
    X = pd.get_dummies(X,"",drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=312148513)
    train_df_scaled = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
    test_df_scaled = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
    
    lm = LinearRegression()  # define our model using least square method
    lm.fit(X_train,y_train)   # Fit our linear model

    #train
    fitted = np.exp(lm.predict(X_train))
    predicted_train = round(pd.Series(fitted, index=y_train.index, name='Predicted_train_'+targets_list_name[idx]),ndigits=2)
    orignal_train=np.exp(train_df_scaled[[targets_list_name[idx]]])
    train_df = pd.merge(left=orignal_train, right=predicted_train, left_index=True, right_index=True)
    train_df

    #test
    fitted = np.exp(lm.predict(X_test))
    predicted_test = round(pd.Series(fitted, index=y_test.index, name='Predicted_test_'+targets_list_name[idx]),ndigits=2)
    orignal_test = np.exp(test_df_scaled[[targets_list_name[idx]]])
    test_df = pd.merge(left=orignal_test, right=predicted_test, left_index=True, right_index=True)
    test_df
     #######################################################################################################################



     
     
     #plot the residuals graph
     #######################################################################################################################
    
     #calculate the residuals
    test_df['residuals'] = test_df['Predicted_test_'+targets_list_name[idx]] - test_df[targets_list_name[idx]]
    train_df['residuals'] = train_df['Predicted_train_'+targets_list_name[idx]] - train_df[targets_list_name[idx]]
    
    fig= go.Figure()
    fig.add_trace(
      go.Scatter(
          x=train_df['Predicted_train_'+targets_list_name[idx]],
          y=train_df.residuals,
          mode='markers',
          name='train residuals',
          marker_color='blue',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
      go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals,
          mode='markers',
          name='test residuals',
          marker_color='red',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.add_trace(
       go.Scatter(
          x=test_df['Predicted_test_'+targets_list_name[idx]],
          y=test_df.residuals*0,
          mode='lines',
          name='zero line',
          marker_color='black',
          marker_size=1.5,
          marker_line_width=0,
       )
     )
    fig.update_layout(
        title="Residuals of Predicted "+targets_list_name[idx],
        xaxis_title="Predicted "+targets_list_name[idx],
        yaxis_title="Residuals",
        font=dict(
            size=14,
            color="RebeccaPurple"
        )
    )
    fig.show()
     #######################################################################################################################
     
     #create folder
     ########################################
    if not os.path.exists(Descripton):
      os.mkdir(Descripton)
    fig.write_image(Descripton+"/results_"+targets_list_name[idx]+'_.png', width=1500, height=600)
    #########################################

    #create txt file with the metrix result 
    ################################################################################################################################################     
    f = open(Descripton+"/results_"+targets_list_name[idx]+'_.txt', 'w')

    #  correct_rows = train_df.loc[abs(train_df['Predicted_train_'+targets_list_name[idx]]-train_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Training = round(correct_rows/len(train_df),3)
    if len(train_df.loc[train_df['Predicted_train_'+targets_list_name[idx]]>=np.infty])/len(train_df)*100 <1:
      train_df.drop(train_df.loc[train_df['Predicted_train_'+targets_list_name[idx]]>=np.infty].index,inplace=True)
    MSE_Training = metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    RMSE_Training = np.sqrt(metrics.mean_squared_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]]))
    MAE_Training = metrics.mean_absolute_error(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    R2_Training=metrics.r2_score(train_df[targets_list_name[idx]], train_df['Predicted_train_'+targets_list_name[idx]])
    std_Training = np.std(predicted_train)

    f.write("------ TRAIN DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Training)+"\n")
    f.write("MSE : "+str(MSE_Training)+", RMSE: "+str(RMSE_Training)+", MAE : "+str(MAE_Training)+"\n")
    f.write("R2 TRAIN "+ str(R2_Training)+"\n")


    #  correct_rows = test_df.loc[abs(test_df['Predicted_test_'+targets_list_name[idx]]-test_df[targets_list_name[idx]])<=50].any(axis=1).count()
    #  ACC_Test = round(correct_rows/len(test_df),3)
    if len(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty])/len(test_df)*100 <1:
      test_df.drop(test_df.loc[test_df['Predicted_test_'+targets_list_name[idx]]>=np.infty].index,inplace=True)   
    MSE_Test = metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    RMSE_Test = np.sqrt(metrics.mean_squared_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]]))
    MAE_Test = metrics.mean_absolute_error(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    R2_Test = metrics.r2_score(test_df[targets_list_name[idx]], test_df['Predicted_test_'+targets_list_name[idx]])
    std_test = np.std(predicted_test)

    f.write("\n")
    f.write("------ TEST DATA ------\n")
    #f.write("Model "+' mod0' +" Accuracy: "+str(ACC_Test)+"\n")
    f.write("MSE : "+str(MSE_Test)+", RMSE: "+str(RMSE_Test)+", MAE : "+str(MAE_Test)+"\n")
    f.write("R2 TEST "+ str(R2_Test)+"\n")
    f.write("--------------------------------\n")

    f.close()
    ################################################################################################################################################     
    coeff = lm.coef_
    equation =[]
    for i in range(len(X.columns)):
         equation.append(((round(coeff[i],4))," * "+str(X.columns[i])))
      
    equation = (sorted(equation, reverse=True,key = lambda x: x[0]))
    equation = " + ".join(map(str,equation))
    equation = ""+ str(lm.intercept_)+" + "+equation
    equation= equation.replace(',', '')
    equation= equation.replace("'", "")
    equation
    #Add to excel result the new line
    ################################################################################################################################################
    Target=targets_list_name[idx]
    Site_in_this_model='ALL'
    new_row = {
      'Descripton':Descripton,
      "Target":Target,
      'Model_type':Model_type,
      # 'Model_number':Model_number,
      'Site_in_this_model':Site_in_this_model,
      "Size Train":len(train_df),
      # 'ACC_Training':ACC_Training,
      'MAE_Training':MAE_Training,
      'MSE_Training':MSE_Training,
      'RMSE_Training':RMSE_Training,
      'STD_Training':std_Training,
      'R2_Training':R2_Training,
      "Size Test":len(test_df),
      # 'ACC_Test':ACC_Test,
      'MAE_Test':MAE_Test,
      'MSE_Test':MSE_Test,
      'RMSE_Test':RMSE_Test,
      'STD_Test':std_test,
      'R2_Test':R2_Test,
      'Check':RMSE_Test<=std_test,
      'equation':equation
      }

    res=res.append(new_row,ignore_index=True)
    ################################################################################################################################################     
    
    # Index for the next target y
    idx+=1

res.to_excel("../../../res.xlsx",index=False)
res.tail(3)

,Descripton,Target,Model_type,Site_in_this_model,Size Train,MAE_Training,MSE_Training,RMSE_Training,STD_Training,R2_Training,Size Test,MAE_Test,MSE_Test,RMSE_Test,STD_Test,R2_Test,Check,equation
423,Only weekend corr drop with treshold 0.8 remov...,Israelis_Count,MLR,ALL,2695,293.465362,286555.590310,535.308874,369.069601,0.341961,1155,297.033810,6.253069e+05,790.763470,359.065768,0.130394,False,-128788202937.95868 + (2731204810486.105 * Gr...
424,Only weekend corr drop with treshold 0.8 remov...,Total,MLR,ALL,2695,325.172260,343395.210338,585.999326,571.242976,0.500740,1153,331.476409,6.784668e+05,823.690952,NaN,0.248520,False,-2259330724690.023 + (1941312216538.7007 * Gr...
425,Only weekend corr drop with treshold 0.8 remov...,Tourists_Count,MLR,ALL,2695,100.176356,95419.268934,308.900095,183.831155,0.077571,1153,144.027875,1.850471e+06,1360.320176,NaN,-20.484635,False,-14009418782018.654 + (18165730138422.83 * Se...


<IPython.core.display.Javascript object>

In [103]:
res.tail(6)

,Descripton,Target,Model_type,Site_in_this_model,Size Train,MAE_Training,MSE_Training,RMSE_Training,STD_Training,R2_Training,Size Test,MAE_Test,MSE_Test,RMSE_Test,STD_Test,R2_Test,Check,equation
402,Only weekend corr drop with treshold 0.6 remov...,Israelis_Count,MLR,ALL,3551,274.429000,236596.544924,486.411909,348.016600,0.341743,1522,305.543666,261120.068015,510.999088,389.378804,0.352488,0.0,-6978824179822.876 + (6978824179827.253 * Gre...
403,Only weekend corr drop with treshold 0.6 remov...,Total,MLR,ALL,3547,309.096665,296446.623518,544.469121,524.552141,0.506964,1521,334.819297,364239.138102,603.522276,547.244008,0.465472,0.0,-5538188044499.587 + (5538188044504.446 * Gre...
404,Only weekend corr drop with treshold 0.6 remov...,Tourists_Count,MLR,ALL,3588,106.002062,83132.375284,288.326855,166.988110,0.112593,1539,100.754912,85705.264321,292.754615,158.074459,0.127050,0.0,9527502694626.664 + (10.2742 * _En Gedi) + (7...
405,Only weekend corr drop with treshold 0.4 remov...,Israelis_Count,MLR,ALL,3551,274.274204,238017.420889,487.870291,334.870721,0.337790,1522,304.853988,264336.689652,514.136839,367.862947,0.344512,False,4.452582452185345 + (3.3507 * _Caesarea) + (2...
406,Only weekend corr drop with treshold 0.4 remov...,Total,MLR,ALL,3547,310.333820,300820.942777,548.471460,511.321675,0.499689,1521,336.770164,367309.637101,606.060754,528.234867,0.460966,False,4.825014303972462 + (3.1519 * _Caesarea) + (2...
407,Only weekend corr drop with treshold 0.4 remov...,Tourists_Count,MLR,ALL,3588,105.931683,83668.418961,289.254938,144.075716,0.106871,1539,102.421650,85716.852921,292.774406,141.062759,0.126932,False,1.085823523055716 + (9.8315 * _En Gedi) + (6....


<IPython.core.display.Javascript object>